In [1]:
from dataset import *
from model import *
from tqdm import tqdm
import time
import datetime
import torch.optim as optim
import torch
import torch.nn as nn
import torchvision
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from omegaconf import OmegaConf

In [2]:
cfg = OmegaConf.load("conf/config.yaml")

In [3]:
resolution = cfg.model.resolution
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
if cfg.dataset.clevr:
    train_set = CLEVR(cfg.dataset.clevr_path, 'train')
    test_set = CLEVR(cfg.dataset.clevr_path, 'test')
    val_set = CLEVR(cfg.dataset.clevr_path, "val")
else:
    MY = MyDataset(cfg.dataset.my_path, cfg.dataset.train_rate, cfg.dataset.test_rate)
    test_set = MY.get_split_data("test")
    train_set = MY.get_split_data("train")
    val_set = MY.get_split_data("val")

In [5]:
def renormalize(x):
    return x / 2. + 0.5


def get_prediction(model2, batch, idx=0):
    recon_combined, recons, masks, slots = model2(batch["image"].to(device))
    image = renormalize(batch["image"].to(device))[idx]
    recon_combined = renormalize(recon_combined)[idx]
    recons = renormalize(recons)[idx]
    masks = masks[idx]
    return image, recon_combined, recons, masks, slots

In [6]:
model = SlotAttentionAutoEncoder(cfg.model.resolution, cfg.slot.num_slots, cfg.slot.num_iterations, cfg.model.hid_dim).to(device)

criterion = nn.MSELoss()

params = [{'params': model.parameters()}]

train_dataloader = torch.utils.data.DataLoader(train_set,
                                               batch_size=cfg.dataset.batch_size,
                                               shuffle=True,
                                               num_workers=cfg.dataset.num_workers)

test_dataloader = torch.utils.data.DataLoader(test_set,
                                              batch_size=cfg.dataset.batch_size,
                                              shuffle=True,
                                              num_workers=cfg.dataset.num_workers)

optimizer = optim.Adam(params, lr=cfg.optim.learning_rate)


In [ ]:
start = time.time()
writer2 = SummaryWriter(cfg.tensorboard.logs_path)
for epoch in range(cfg.param.num_epochs):
    model.train()

    total_loss = 0

    for i, sample in tqdm(enumerate(train_dataloader, 1),
                          total=len(train_dataloader)):
        
        if i < cfg.param.warmup_steps:
            #学習率のWarmup
            learning_rate = cfg.optim.learning_rate * (i / cfg.param.warmup_steps)
        else:
            # ウォームアップ後の学習率
            learning_rate = cfg.optim.learning_rate
        # 指数的な学習率減衰
        learning_rate = learning_rate * (cfg.param.decay_rate ** (
            i / cfg.param.decay_steps))

        optimizer.param_groups[0]['lr'] = learning_rate
        image = sample['image'].to(device)
        recon_combined, recons, masks, slots = model(image)
        loss = criterion(recon_combined, image)
        total_loss += loss.item()

        del recons, masks, slots

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss /= len(train_dataloader)

    print("Epoch: {}, Loss: {}, Time: {}". format(epoch, total_loss, datetime.timedelta(seconds=time. time() - start)))
    writer2.add_scalar("loss", total_loss, epoch)
    writer2.add_scalar("lr", learning_rate, epoch)
    if not epoch % 10:
        torch.save({
            'model_state_dict': model.state_dict(),
            }, cfg.model.model_dir)
        writer = SummaryWriter(cfg.tensorboard.img_path+str(epoch))
        dataiter = iter(test_dataloader)
        images = next(dataiter)
        model.eval()
        image, recon_combined, recons, masks, slots = get_prediction(model, images)
        num_slots = len(masks)
        image = torchvision.utils.make_grid(image.to("cpu"), nrow=1)
        writer.add_image('Slot_images', image)
        writer.add_image('Recon', recon_combined.to("cpu").detach().numpy())
        for i in range(num_slots):
            s_m = np.clip((recons[i].to("cpu").detach().numpy()*masks[i].to("cpu").detach().numpy())+(1-masks[i].to("cpu").detach().numpy()),0, 1)
            writer.add_image('Slot '+str(i+1), torch.from_numpy(s_m.astype(np.float32)).permute(2,0,1))

100%|███████████████████████████████████████| 4375/4375 [10:48<00:00,  6.75it/s]


Epoch: 0, Loss: 1.2774966580308282e-06, Time: 0:10:48.686520


100%|███████████████████████████████████████| 4375/4375 [10:46<00:00,  6.76it/s]

Epoch: 1, Loss: 9.516004337700822e-07, Time: 0:21:37.495470



100%|███████████████████████████████████████| 4375/4375 [10:46<00:00,  6.76it/s]

...


100%|███████████████████████████████████████| 4375/4375 [10:43<00:00,  6.80it/s]

Epoch: 68, Loss: 2.4545200610543013e-07, Time: 12:21:18.337697



100%|███████████████████████████████████████| 4375/4375 [10:43<00:00,  6.80it/s]

Epoch: 69, Loss: 2.1377653620490035e-07, Time: 12:32:01.723006



 50%|███████████████████▌                   | 2199/4375 [05:23<05:20,  6.78it/s]